In [1]:
import os
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import random
import platform
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import importlib
from dataclasses import asdict
from loguru import logger
import sys

from reload_recursive import reload_recursive

import mri_data
import monai_training

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.4.0
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: 2.18.0
gdown version: 5.2.0
TorchVision version: 0.20.1+cu124
tqdm version: 4.66.6
lmdb version: 1.5.1
psutil version: 6.1.0
pandas version: 2.2.3
einops version: 0.8.0
transformers version: 4.46.2
mlflow version: 2.17.2
pynrrd version: 1.1.1
clearml version: 1.16.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [2]:
from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training, preprocess

In [3]:
logger.remove()

In [4]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
curr_dir = msmri_home / "analysis" / "choroid_pineal_pituitary_crosstrain_t1"
training_work_dirs = msmri_home / "training_work_dirs"
modalities = ["t1"]
labels = ["choroid_t1_flair", "pineal", "pituitary"]

### First Set

In [5]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary_T1-1"
work_dir = training_work_dirs / work_dir_name

In [6]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset1.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 384.87it/s]


In [7]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [8]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [9]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-14 17:46:24,705 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1
2025-01-14 17:46:24,737 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/datalist.json.
2025-01-14 17:46:24,737 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/datalist.json.
2025-01-14 17:46:24,748 - INFO - Using user defined command running prefix , will override other settings


In [10]:
runner.run()

2025-01-14 17:46:24,758 - INFO - Skipping data analysis...
2025-01-14 17:46:24,759 - INFO - Skipping algorithm generation...
2025-01-14 17:46:24,780 - INFO - ['python', '/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_0/scripts/train.py', 'run', "--config_file='/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_0/configs/hyper_parameters.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_0/configs/network.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_0/configs/transforms_infer.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_0/configs/transforms_train.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_0/configs/transforms_validate.yaml'", '--num_epochs_per_validation=1', '--num_epochs=100', '--num_warmup_epochs=1']


monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-14 17:46:28,792 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_0/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.
2025-01-14 17:46:28,792 - INFO - File exists: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_0/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt, skipped downloading.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/14 17:46:28 INFO mlflow.

2025-01-14 18:06:08,527 - INFO - ['python', '/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_1/scripts/train.py', 'run', "--config_file='/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_1/configs/hyper_parameters.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_1/configs/network.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_1/configs/transforms_infer.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_1/configs/transforms_train.yaml,/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_1/configs/transforms_validate.yaml'", '--num_epochs_per_validation=1', '--num_epochs=100', '--num_warmup_epochs=1']


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-14 19:12:59,567 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-14 19:12:59,567 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/14 19:12:59 INFO mlflow.

2025-01-14 19:36:00,697 - INFO - Ensembling using single GPU!
2025-01-14 19:36:00,698 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/ensemble_output will be used to save ensemble predictions.
2025-01-14 19:36:00,698 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/ensemble_output is created to save ensemble predictions
2025-01-14 19:36:00,731 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-14 19:36:00,731 - INFO - swinunetr_0
2025-01-14 19:36:00,731 - INFO - swinunetr_1
2025-01-14 19:36:00,732 - INFO - swinunetr_2
2025-01-14 19:36:00,732 - INFO - swinunetr_3
2025-01-14 19:36:00,732 - INFO - swinunetr_4
2025-01-14 19:36:00,732 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-1/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Second Set

In [11]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary_T1-2"
work_dir = training_work_dirs / work_dir_name

In [12]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset2.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 269.17it/s]


In [13]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [14]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [15]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-14 19:44:16,342 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2
2025-01-14 19:44:16,343 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2/datalist.json.
2025-01-14 19:44:16,343 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2/datalist.json.
2025-01-14 19:44:16,344 - INFO - Using user defined command running prefix , will override other settings


In [16]:
runner.run()

2025-01-14 19:44:16,353 - INFO - Running data analysis...


2025-01-14 19:44:16,354 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:12<00:00,  2.39it/s]

2025-01-14 19:44:28,946 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-14 19:44:28,946 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2/datastats.yaml.
2025-01-14 19:44:28,952 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2/datastats_by_case.yaml, this may take a while.


2025-01-14 19:44:29,076 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-14 21:20:38,228 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-14 21:20:38,228 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/14 21:20:38 INFO mlflow.

2025-01-14 21:40:39,529 - INFO - Ensembling using single GPU!
2025-01-14 21:40:39,529 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2/ensemble_output will be used to save ensemble predictions.
2025-01-14 21:40:39,530 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2/ensemble_output is created to save ensemble predictions
2025-01-14 21:40:39,554 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-14 21:40:39,555 - INFO - swinunetr_0
2025-01-14 21:40:39,555 - INFO - swinunetr_1
2025-01-14 21:40:39,555 - INFO - swinunetr_2
2025-01-14 21:40:39,555 - INFO - swinunetr_3
2025-01-14 21:40:39,555 - INFO - swinunetr_4
2025-01-14 21:40:39,556 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-2/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Third Set

In [17]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary_T1-3"
work_dir = training_work_dirs / work_dir_name

In [18]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset3.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 263.21it/s]


In [19]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [20]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [21]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-14 21:48:50,128 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3
2025-01-14 21:48:50,129 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3/datalist.json.
2025-01-14 21:48:50,130 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3/datalist.json.
2025-01-14 21:48:50,130 - INFO - Using user defined command running prefix , will override other settings


In [22]:
runner.run()

2025-01-14 21:48:50,140 - INFO - Running data analysis...


2025-01-14 21:48:50,140 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:11<00:00,  2.53it/s]

2025-01-14 21:49:02,038 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-14 21:49:02,039 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3/datastats.yaml.
2025-01-14 21:49:02,044 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3/datastats_by_case.yaml, this may take a while.


2025-01-14 21:49:02,170 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-14 23:20:58,324 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-14 23:20:58,324 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/14 23:20:58 INFO mlflow.

2025-01-14 23:39:10,389 - INFO - Ensembling using single GPU!
2025-01-14 23:39:10,390 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3/ensemble_output will be used to save ensemble predictions.
2025-01-14 23:39:10,390 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3/ensemble_output is created to save ensemble predictions
2025-01-14 23:39:10,415 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-14 23:39:10,416 - INFO - swinunetr_0
2025-01-14 23:39:10,416 - INFO - swinunetr_1
2025-01-14 23:39:10,416 - INFO - swinunetr_2
2025-01-14 23:39:10,416 - INFO - swinunetr_3
2025-01-14 23:39:10,416 - INFO - swinunetr_4
2025-01-14 23:39:10,417 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-3/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Fourth Set

In [23]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary_T1-4"
work_dir = training_work_dirs / work_dir_name

In [24]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset4.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 251.68it/s]


In [25]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [26]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [27]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-14 23:47:31,117 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4
2025-01-14 23:47:31,118 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4/datalist.json.
2025-01-14 23:47:31,118 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4/datalist.json.
2025-01-14 23:47:31,119 - INFO - Using user defined command running prefix , will override other settings


In [28]:
runner.run()

2025-01-14 23:47:31,128 - INFO - Running data analysis...


2025-01-14 23:47:31,128 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:12<00:00,  2.41it/s]

2025-01-14 23:47:43,627 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-14 23:47:43,627 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4/datastats.yaml.
2025-01-14 23:47:43,633 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4/datastats_by_case.yaml, this may take a while.


2025-01-14 23:47:43,957 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-15 01:13:36,438 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-15 01:13:36,438 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/15 01:13:36 INFO mlflow.

2025-01-15 01:39:01,559 - INFO - Ensembling using single GPU!
2025-01-15 01:39:01,559 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4/ensemble_output will be used to save ensemble predictions.
2025-01-15 01:39:01,560 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4/ensemble_output is created to save ensemble predictions
2025-01-15 01:39:01,584 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-15 01:39:01,584 - INFO - swinunetr_0
2025-01-15 01:39:01,584 - INFO - swinunetr_1
2025-01-15 01:39:01,585 - INFO - swinunetr_2
2025-01-15 01:39:01,585 - INFO - swinunetr_3
2025-01-15 01:39:01,585 - INFO - swinunetr_4
2025-01-15 01:39:01,585 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary_T1-4/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w